In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import ast
import json
from collections import defaultdict
import csv

In [31]:
def parse_complex_value(value):
    try:
        # Remove 'array([...], dtype=int16)' patterns
        value = re.sub(r"array\((\[.*?\])\s*,\s*dtype=int16\)", r"\1", value)
        # Replace single quotes with double quotes
        value = value.replace("'", '"')
        # Evaluate the string as a Python literal
        return ast.literal_eval(value)
    except Exception as e:
        print(f"Error parsing value: {e}")
        return None

def parse_csv(file_path):
    results = []
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            probe_name, visual_area, iteration, iteration_phi_values = row
            phi_values = parse_complex_value(iteration_phi_values)
            
            if phi_values is None:
                print(f"Skipping row due to parsing error: {row}")
                continue

            for presentation_type, intervals in phi_values.items():
                for interval, data in intervals.items():
                    if isinstance(interval, tuple):
                        interval_start, interval_end = interval
                    else:
                        interval = interval.strip('()')
                        interval_start, interval_end = map(int, interval.split(','))
                    
                    if len(data) >= 3:
                        max_phi, max_phi_state, metrics = data
                    else:
                        print(f"Data tuple does not have 3 elements: {data}")
                        continue

                    results.append({
                        'probe_name': probe_name,
                        'visual_area': visual_area,
                        'presentation_type': presentation_type,
                        'interval_start': interval_start,
                        'interval_end': interval_end,
                        'max_phi': max_phi,
                        'max_phi_state': str(max_phi_state),
                        'total_distinctions': metrics['total_distinctions'],
                        'total_integrated_information_distinctions': metrics['total_integrated_information_distinctions'],
                        'total_cause_information': metrics['total_cause_information'],
                        'total_effect_information': metrics['total_effect_information'],
                        'total_relations': metrics['total_relations'],
                        'total_integrated_information_relations': metrics['total_integrated_information_relations'],
                        'faces_1_9': metrics['faces_1_9'],
                        'faces_10_30': metrics['faces_10_30'],
                        'faces_30_plus': metrics['faces_30_plus']
                    })
        
    return results

def process_directory(directory_path):
    all_results = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            all_results.extend(parse_csv(file_path))
    return all_results

def write_output(results, output_dir):
    output_file = os.path.join(output_dir, 'parsed_probe_data.csv')
    fieldnames = [
        'probe_name', 'visual_area', 'presentation_type', 'interval_start', 'interval_end',
        'max_phi', 'max_phi_state', 'total_distinctions', 'total_integrated_information_distinctions',
        'total_cause_information', 'total_effect_information', 'total_relations',
        'total_integrated_information_relations', 'faces_1_9', 'faces_10_30', 'faces_30_plus'
    ]
    
    with open(output_file, 'w', newline='', encoding='ISO-8859-1') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in results:
            writer.writerow(row)

In [32]:
input_directory = '/Users/paxpoggi/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maier Lab/IIT Notebooks/phi_values_lfp_hypergraphs/'
output_directory = '/Users/paxpoggi/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maier Lab/IIT Notebooks/Cleaned Graph Data/'

results = process_directory(input_directory)
write_output(results, output_directory)
print(f"Parsing complete. Output written to {os.path.join(output_directory, 'parsed_probe_data_full.csv')}")

Parsing complete. Output written to /Users/paxpoggi/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maier Lab/IIT Notebooks/Cleaned Graph Data/parsed_probe_data_full.csv
